In [1]:
import uproot as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
run="run1"
dm_type = 'fermion'
target_pot=2e20
signal_pot={}

signal_correction = { "0.01":0.94903385, "0.02":1.03561568, "0.03":1.02326517, "0.04":1.00641579, 
                     "0.05":1.00650082, "0.06":1.01293082, "0.07":1.07433777, 
                     "0.08":1.10175345, "0.09":1.0415591 , "0.10":1.1014797}

if(dm_type == "fermion"):
    signal_pot = { "0.01":4.49230189e+20, "0.02":3.24986846e+21, "0.03":1.44413783e+22, 
                  "0.04":5.51306150e+22, "0.05":1.74805153e+23, "0.06":5.79056950e+23,
                  "0.07":1.48783150e+24, "0.08":4.41322940e+24, 
                  "0.09":9.92424735e+24, "0.10":2.27414738e+25}

else: 
    signal_pot = { "0.01":8.70545727e+21, "0.02": 9.48797710e+22, "0.03":4.86277630e+23, 
                  "0.04":1.94231827e+24, "0.05":8.19564834e+24, "0.06":2.06542023e+25,
                  "0.07":5.83217654e+25, "0.08":1.54999742e+26, 
                  "0.09":2.94329651e+26, "0.10":5.04481005e+26}



In [3]:
def logit_transform(score):
    return np.log(score/(1-score))

def filter_df(df):
    df = df[df['signal_score'] >= 0.5]
    return df 

In [4]:
def MakeSignalHist():
    base_dir = "/home/lmlepin/Desktop/dm_sets/dark_tridents_analysis/" + run + "_samples/"
    signal_file = u.recreate(base_dir + dm_type + "_signal_hist_" + run + "_CNN.root")
    masses = ["0.01", "0.02", "0.03", "0.04", "0.05", "0.06", "0.07", "0.08", "0.09", "0.10"]
    for mass in masses:
        signal_scaling = signal_correction[mass]*target_pot/signal_pot[mass]
        df_signal = pd.read_csv(base_dir + "dt_overlay_"+ mass + "_CNN_scores_8441_steps.csv")
        df_signal = filter_df(df_signal)
        df_signal['signal_score'] = logit_transform(df_signal['signal_score'])
        my_custom_bins = [ 0., 1.375, 2.75, 4.125, 5.5, 11.] # Last four bins merged bc of low stats
        (counts_signal, bins_signal) = np.histogram(df_signal['signal_score'], bins = my_custom_bins )
        signal_file["signal_"+mass] = (counts_signal*signal_scaling, bins_signal)

In [5]:
MakeSignalHist()